In [ ]:
basic_feature_list = []
for link in all_links:
    
    # opening urls
    bs_object = BeautifulSoup(requests.get(link, headers=headers).text, "html.parser")
    
    #neighbourhood family vs single ratio
    neighb_family = bs_object.find("span", {"class": "css-1ruej2h"})
    neighb_single = bs_object.find("span", {"class": "css-1ymya4c"})
    
    
    #street owner vs renter
    street_renter = bs_object.find("span", {"class": "css-1ejmyle"})
    street_owner = bs_object.find("span", {"class": "css-1jhlvvv"})

    
    #long term resident percentage
    long_term_resi= bs_object.find("div", {"class": "css-ibsnk8"})
    
    # extracting property price
    property_price = bs_object.find("div", {"data-testid": "listing-details__summary-title"})
    
    
    
    
    
    # dictionary to store temporary data
    basic_feature_dict = {}
    
    # putting 'none' if recent_rented number is missing   
    if recent_rented_number is None:
        basic_feature_dict['recent_rented_number'] = None
        
    else:
        basic_feature_dict['recent_rented_number'] = pattern1.findall(str(recent_rented_number))[0]
        
        
    # putting 'none' if price is missing    
    if property_price is None:
        basic_feature_dict['price'] = None
        
    else:
        basic_feature_dict['price'] = pattern1.findall(str(property_price))[0]
        

In [ ]:
# appending all the data into a list
basic_feature_list.append(basic_feature_dict)

In [ ]:
for row in basic_feature_list:
    # checking if the row cointains 'price'         
    if 'price' in row:
        price_list.append(row['price'])
    else:
        price_list.append(None)        
        
    # checking if the row cointains 'recent_rented_number'            
    if 'recent_rented_number' in row:
        recent_rented_number_list.append(row['recent_rented_number'])
    else:
        recent_rented_number_list.append(None)
    
        
    # checking if the row cointains 'long_term_resi_percentage'            
    if 'long_term_resi_percentage' in row:
        long_term_resi_percentage_list.append(row['long_term_resi_percentage'])
    else:
        long_term_resi_percentage_list.append(None)
        
    # checking if the row cointains 'street_renter_owner_ratio'            
    if 'street_renter_owner_ratio' in row:
        street_renter_owner_ratio_list.append(row['street_renter_owner_ratio'])
    else:
        street_renter_owner_ratio_list.append(None)
        
    # checking if the row cointains 'neighb_family_single_ratio'            
    if 'neighb_family_single_ratio' in row:
        neighb_family_single_ratio_list.append(row['neighb_family_single_ratio'])
    else:
        neighb_family_single_ratio.append(None)

In [ ]:
house_dict['Rent'] = price_list
house_dict['street_renter_owner_ratio'] = street_renter_owner_ratio_list 
house_dict['neighb_family_single_ratio'] = neighb_family_single_ratio_list
house_dict['long_term_resi_percentage'] = long_term_resi_percentage_list
house_df = pd.DataFrame(house_dict)

In [ ]:
#find pattern for week/annual rent
year_pattern = r"p.a.|annually|annual"
week_pattern = r"pw|week"

regex_week_pattern = re.compile(week_pattern)
regex_year_pattern = re.compile(year_pattern)

#flag type of rent with 1(weekly), 0(annul), None(unknown)
def type_flag(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return 1
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return 0
    # not speficied
    else:
        return None

#convert rent to weekly rent
#if not sure rent type, it will be considered as weekly rent
def convert_rent(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return remove_chars(s)
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return float(remove_chars(s))//12
    # not speficied
    else:
        return remove_chars(s)

In [ ]:
house_df['Converted_Rent'] = [convert_rent(i) for i in price_list]
house_df['Type'] = [type_flag(i) for i in price_list]